In [21]:
import numpy as np
import matplotlib.pyplot as plt
import sounddevice as sd
import pandas as pd

In [38]:
# Set the sampling frequency
Fs = 44100

# Read first row of the text file containing the synthesis parameters using numpy
data = np.loadtxt('g1_s1_beed_est_l.txt', max_rows=1)

Nf, n = data[:-1].astype(int)
dt = data[-1].astype(float)

# Get the amplitude and frequency values and store in an ndarray
data = np.loadtxt('g1_s1_beed_est_l.txt', skiprows=1)

# Get the amplitude and frequency values and store in an ndarray
an = data[0::2, :].astype(float)
fn = data[1::2, :].astype(float)


In [39]:
def sos_interp(a, f, Fs, dt):
    """
        Interpolates the signal values of an input partial from its amplitude and frequency values
        Adapted from sosinterp.m by Franklin Agsaway, 2005
    """

    NperFrame = round(dt * Fs)      # number of samples per frame (original sample length per frame == 44100 * 0.015986 = 705 samples)
    dt = NperFrame / Fs
    Nf = len(a)     # number of frames 

    dA = np.append(np.diff(a), -a[-1]) / NperFrame     # amplitude increment per sample 
    # Amplitude synthesis interpolation
    asyn = np.zeros(Nf * NperFrame)    # Each row in Ats represents the interpolated amp values of each partial
    asyn[::NperFrame] = a
    for i in range(1, NperFrame):
        asyn[i::NperFrame] = asyn[(i-1)::NperFrame] + dA
    # Frequency synthesis interpolation
    fsyn = np.zeros(NperFrame * Nf)  # Each row in Fts represents the frame-varying freq values of each partial
    for i in range(NperFrame):
        fsyn[i::NperFrame] = f
    # Phase synthesis interpolation
    startphase = 0
    p = np.zeros(Nf)
    for i in range(Nf):
        p[i] = startphase - 2 * np.pi * ((i) * dt) * f[i]
        startphase += 2 * np.pi * dt * f[i]

    psyn = np.zeros(NperFrame * Nf)  # Each row in Fts represents the frame-varying freq values of each partial
    for i in range(NperFrame):
        psyn[i::NperFrame] = p

    t = np.arange(0, dt * Nf, 1 / Fs)
    sig = asyn * np.sin(2 * np.pi * fsyn * t + psyn)

    return sig



In [40]:
# sos
sig = np.zeros(71910) 
for i in range(1, len(an)):
    partial = sos_interp(an[i], fn[i], Fs, dt)
    sig = sig + partial


In [44]:
sd.play(sig, Fs)

(array([13]), array([1]))
0.585916
0.585916


In [50]:
a = an[13]
f = fn[13]

NperFrame = round(dt * Fs)      # number of samples per frame (original sample length per frame == 44100 * 0.015986 = 705 samples)
dt = NperFrame / Fs
Nf = len(a)     # number of frames 

dA = np.append(np.diff(a), -a[-1])
dA = dA / NperFrame     # amplitude increment per sample 
# Amplitude synthesis interpolation
asyn = np.zeros(Nf * NperFrame)    # Each row in Ats represents the interpolated amp values of each partial
asyn[::NperFrame] = a
for i in range(1, NperFrame):
    asyn[i::NperFrame] = asyn[(i-1)::NperFrame] + dA
# Frequency synthesis interpolation
fsyn = np.zeros(NperFrame * Nf)  # Each row in Fts represents the frame-varying freq values of each partial
for i in range(NperFrame):
    fsyn[i::NperFrame] = f
# Phase synthesis interpolation
startphase = 0
p = np.zeros(Nf)
for i in range(Nf):
    p[i] = startphase - 2 * np.pi * ((i) * dt) * f[i]
    startphase += 2 * np.pi * dt * f[i]

psyn = np.zeros(NperFrame * Nf)  # Each row in Fts represents the frame-varying freq values of each partial
for i in range(NperFrame):
    psyn[i::NperFrame] = p

t = np.arange(0, dt * Nf, 1 / Fs)
sig = asyn * np.sin(2 * np.pi * fsyn * t + psyn)

In [53]:
a = an[13]
f = fn[13]

NperFrame = round(dt * Fs)      # number of samples per frame (original sample length per frame == 44100 * 0.015986 = 705 samples)
dt = NperFrame / Fs
Nf = len(a)     # number of frames

dA = np.append(np.diff(a), -a[-1]) / NperFrame     # amplitude increment per sample
 
t = np.arange(0, dt * Nf, 1 / Fs)
asyn = np.tile(a, NperFrame) + np.repeat(np.cumsum(np.tile(dA, Nf)), NperFrame)
fsyn = np.repeat(f, NperFrame)
psyn = np.repeat(np.cumsum(2 * np.pi * dt * f), NperFrame)

sig = asyn * np.sin(2 * np.pi * fsyn * t + psyn)

ValueError: operands could not be broadcast together with shapes (71910,) (7262910,) 